The purpose of the project is to create a machine learning program that can accurately recommend a data plan that best fits the user's behavior.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error


df = pd.read_csv(r'C:\Users\Justin\Downloads\data\TT projects\7\data raw.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.isnull().any().any()

False

In [4]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Data set is complete.


In [5]:
medians_by_plan = df.groupby('is_ultra').median()
print(medians_by_plan)

          calls  minutes  messages   mb_used
is_ultra                                    
0          60.0   410.56      28.0  16506.93
1          74.0   502.55      38.0  19308.01


In [6]:
means_by_plan = df.groupby('is_ultra').mean()
print(means_by_plan)

              calls     minutes   messages       mb_used
is_ultra                                                
0         58.463437  405.942952  33.384029  16208.466949
1         73.392893  511.224569  49.363452  19468.823228


In [7]:
ultra_count = df[df['is_ultra'] == 1].count()
print(ultra_count)

calls       985
minutes     985
messages    985
mb_used     985
is_ultra    985
dtype: int64


In [8]:
smart_count = df[df['is_ultra'] == 0].count()
print(smart_count)

calls       2229
minutes     2229
messages    2229
mb_used     2229
is_ultra    2229
dtype: int64


In [9]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

First Split

In [10]:
features_train_val, features_test, target_train_val, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

In [11]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_val, target_train_val, test_size=0.25, random_state=12345)

In [14]:
for depth in range(1, 6):
    modelDT = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    modelDT.fit(features_train_val, target_train_val)
    predictions_valid = modelDT.predict(features_valid)
    print('max_depth =', depth, ': ', end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7418351477449455
max_depth = 2 : 0.7651632970451011
max_depth = 3 : 0.7729393468118196
max_depth = 4 : 0.7822706065318819
max_depth = 5 : 0.7884914463452566


In [1]:
best_score = 0
best_est = 0
for est in range(1, 5):
    modelRF = RandomForestClassifier(random_state=54321, n_estimators=est)
    modelRF.fit(features_train, target_train)
    score = modelRF.score(features_valid, target_valid)
    if score > best_score:
        best_score = score
        best_est = est

print("Accuracy of the best model on the validation set (n_estimators = {}): {}".format(
    best_est, best_score))

NameError: name 'RandomForestClassifier' is not defined

In [ ]:
modelLR = LogisticRegression(random_state=54321, solver="liblinear")
modelLR.fit(features_valid, target_valid)
score_train = modelLR.score(features_valid, target_valid)

print("Accuracy of the logistic regression model on the training set:", score_train,)

Accuracy of the logistic regression model on the training set: 0.7107309486780715


Best set is RandomForest at 0.95877

In [ ]:
final_model = RandomForestClassifier(random_state=54321, n_estimators=4)
final_model.fit(features_test, target_test)
test_predictions = final_model.predict(features_test)

print('Test set Random Forest:', accuracy_score(target_test, test_predictions))

Test set Random Forest: 0.9331259720062208


RandomForest as described should have the highest accuracy as it does here the speed is negligible as the size of the dataset does not have a meaningful effect. There is a 5% marign of error which could be improved but is acceptable. 